In [1]:
import re
import math
import pandas as pd
import csv

import jieba
from collections import OrderedDict

from pprint import pprint

In [2]:
jobs7 = pd.read_csv('job7_4groups.csv', encoding='utf-8-sig')

In [3]:
text = pd.DataFrame({
    'category': jobs7["職位_"],
    'content': list(jobs7["工作內容"] + ' ' + jobs7["其他條件"] + ' ' + jobs7["工作技能"])
})

In [4]:
text.head()

,category,content
0,職位 IV,1.系統功能需求分析設計。\n2.系統功能規劃與架構建構佈署。\n3.具出色的溝通能力與組織...
1,職位 IV,1、負責NBS升級案之專案執行之文件與進度管理2、監督工作成員的各項工作進度回報3、與客戶進...
2,職位 IV,VICI Holdings 威旭資訊是一間專注於高頻、造市及套利交易的公司，我們進行量化研究...
3,職位 IV,1. 熟悉反洗錢架構2. 熟悉SAS AML系統3. 有帶領團隊經驗4. 善溝通、有技術程式...
4,職位 IV,洞察和量化系統流程瓶頸，關注服務穩定性 發掘系統 performance 和 secur...


In [6]:
text = text.dropna().reset_index()
len(text)

56392

In [7]:
text = text.groupby(['category'])['content'].apply(lambda x: ' '.join(x)).reset_index()
text

,category,content
0,職位 I,1.對於微控制器程式設計有興趣的人員※工作內容：1. 量測、控制、通訊及人機介面等韌體單元設...
1,職位 II,1.負責網路安控軟體系統開發、系統測試2.熟PKI、專案管理、Java、Asp、Jsp、Ja...
2,職位 III,1.IC驗證 \n2.驗證系統開發 \n3.客戶design in問題解決 \n4.FPGA...
3,職位 IV,1.系統功能需求分析設計。\n2.系統功能規劃與架構建構佈署。\n3.具出色的溝通能力與組織...


In [8]:
#拔符號

In [9]:
r_1 = r'https?://[a-zA-Z0-9./_%-@=]+|[a-zA-Z0-9]+@[a-zA-Z0-9.]+|@[a-zA-Z0-9.]+|\d\.|\(\d\)'
r_2 = r'，|,|。|…|！|!|？|\?|、|､|；|【|】|〖|〗|「|」|｢|:|：|;|；|＆|&|\(|（|\)|）|{|}|・|．|<|>|｜|_|-|—|–|╴|￣|\[|]|［|］|\*|%|~|～|\'|\"|{|}|｛|｝|\^|⌃|\$|\||●|•|·|°|★|✦|❖|￭|■|◆|※|✓|√|⇧|⌘|＝|=|@|ž|\r'

In [11]:
content1 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for i in range(len(text)):
    c00 = re.sub(r_1, ' ', text.loc[i, 'content'])
    c01 = re.sub(r_2, ' ', c00)
    c02 = re.sub(r'^\s+', '', c01)
    content1.loc[i, 'content'] = c02

In [12]:
content1

,category,content
0,職位 I,對於微控制器程式設計有興趣的人員 工作內容 量測 控制 通訊及人機介面等韌體單元設計撰寫...
1,職位 II,負責網路安控軟體系統開發 系統測試 熟PKI 專案管理 Java Asp Jsp Java ...
2,職位 III,IC驗證 \n 驗證系統開發 \n 客戶design in問題解決 \n FPGA驗證 熟電...
3,職位 IV,系統功能需求分析設計 \n 系統功能規劃與架構建構佈署 \n 具出色的溝通能力與組織協調能力...


In [13]:
#拔停頓詞

In [14]:
stop = r'的|你|妳|我|他|們|您|這個|那個|與|和|或|及|之|等|都|不只|是|以及|每次|提供|例如|並|者|又|至|最|於|需|需要|而且|且|目前|尋找|想找|尤佳|更|佳|希望|職缺|公司|工作|內容|職務|面試|面談|人才|履歷|職位|包含|一間|具備|熟悉|熟練|成立|主要|單位|眾多|負責|歡迎|加入|至少|使用|相關|擔任|回覆|對於|協助|辦理|事務|執行|關於|以下|其他|致力|身為|簡介|企業|另行|懂|非常|謝謝|帶來|若|如果|如下|未填寫|不拘|獎金|加班|薪資|補助|第一'
stop_eng = r'\ba\b|\bi\b|\byou\b|\bwe\b|\bis\b|\bare\b|\bwas\b|\bwere\b|\band\b|\bthe\b|\bin\b|\bfor\b|\bon\b|\bto\b|\bat\b|\bof\b|\bfujitsu\b|\bit\b|\bour\b|\byour\b|\bmy\b|\bby\b|\bthat\b|\bwith\b|\bas\b|\ban\b|\bteam\b|\bwill\b|\bor\b|\bbe\b'
stop_jap = r'[ぁ-んァ-ンｧ-ﾝﾞﾟ｡]'

In [15]:
content2 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for i in range(len(content1)):
    c10 = re.sub(stop, ' ', content1.loc[i, 'content'])
    c11 = c10.lower()
    c12 = re.sub(stop_eng, ' ', c11)
    c13 = re.sub(stop_jap, ' ', c12)
    c14 = re.sub(r'\s+', ' ', c13)
    content2.loc[i, 'content'] = c14

In [16]:
content2

,category,content
0,職位 I,微控制器程式設計有興趣 人員 量測 控制 通訊 人機介面 韌體單元設計撰寫 驗證 配合韌體...
1,職位 II,網路安控軟體系統開發 系統測試 熟pki 專案管理 java asp jsp java s...
2,職位 III,ic驗證 驗證系統開發 客戶design in問題解決 fpga驗證 熟電子電路原理 任一技...
3,職位 IV,系統功能 求分析設計 系統功能規劃 架構建構佈署 具出色 溝通能力 組織協調能力 邏輯分析能...


In [17]:
#切關鍵字

In [20]:
jieba.add_word('asp.net')
jieba.add_word('區塊鍊')
jieba.add_word('區塊鏈')
jieba.add_word('加分項')
jieba.add_word('加分')
jieba.add_word('sas')
jieba.add_word('aml')
jieba.add_word('es6+')
jieba.add_word('信託')
jieba.add_word('大數據')
jieba.add_word('撰寫')
jieba.add_word('身體')
jieba.add_word('公佈')
jieba.add_word('方向')
jieba.add_word('吸引力')
jieba.add_word('為什麼')
jieba.add_word('製作')
jieba.add_word('mysql')
jieba.add_word('mssql')
jieba.add_word('sqlite')
jieba.add_word('rfp')
jieba.add_word('新創')
jieba.add_word('工程師')
jieba.add_word('帶領')
jieba.add_word('佈署')
jieba.add_word('製作')
jieba.add_word('追蹤')
jieba.add_word('開發')
jieba.add_word('經驗')
jieba.add_word('優化')
jieba.add_word('作業')
jieba.add_word('透過')
jieba.add_word('程式碼')
jieba.add_word('回應')
jieba.add_word('不確定')
jieba.add_word('委託')
jieba.add_word('接下來')
jieba.add_word('壓力測試')
jieba.add_word('沒有')
jieba.add_word('比特幣')
jieba.add_word('彼特幣')
jieba.add_word('最大值')
jieba.add_word('最小值')
jieba.add_word('訓練集')
jieba.add_word('邏輯式迴歸')
jieba.add_word('邏輯式回歸')
jieba.add_word('羅吉斯迴歸')
jieba.add_word('羅吉斯回歸')
jieba.add_word('迴歸')
jieba.add_word('線性迴歸')
jieba.add_word('線性回歸')
jieba.add_word('特徵')
jieba.add_word('程式碼')
jieba.add_word('物件導向')
jieba.add_word('前端')
jieba.add_word('後端')
jieba.add_word('全端')
jieba.add_word('前後端')
jieba.add_word('工程師')
jieba.add_word('分類器')
jieba.add_word('機器學習')
jieba.add_word('深度學習')
jieba.add_word('演算法')
jieba.add_word('監督式')
jieba.add_word('機率')
jieba.add_word('人工智慧')
jieba.add_word('決策樹')
jieba.add_word('數據分析')
jieba.add_word('資料探勘')
jieba.add_word('正規化')
jieba.add_word('過度配適')
jieba.add_word('特徵工程')
jieba.add_word('特徵選取')
jieba.add_word('svm')
jieba.add_word('random forest')
jieba.add_word('隨機森林')
jieba.add_word('sklearn')
jieba.add_word('矩陣')
jieba.add_word('自然語言處理')
jieba.add_word('logistic regression')
jieba.add_word('linear regression')
jieba.add_word('k means')
jieba.add_word('主成分分析')
jieba.add_word('統計')
jieba.add_word('maximum likelihood estimation')
jieba.add_word('reinforcement learning')
jieba.add_word('decision tree')
jieba.add_word('id3')
jieba.add_word('naive bayes')
jieba.add_word('polynomial regression')
jieba.add_word('lasso regression')
jieba.add_word('ridge regression')
jieba.add_word('elasticnet regression')
jieba.add_word('網頁設計')
jieba.add_word('專案管理')
jieba.add_word('梯度下降法')
jieba.add_word('資安')
jieba.add_word('word embedding')
jieba.add_word('網路爬蟲')
jieba.add_word('資工')
jieba.add_word('叢集')
jieba.add_word('迴歸樹')
jieba.add_word('回歸樹')
jieba.add_word('分類樹')
jieba.add_word('管理員')
jieba.add_word('系統')
jieba.add_word('安裝')
jieba.add_word('邏輯斯迴歸')
jieba.add_word('邏輯斯回歸')
jieba.add_word('裝置')
jieba.add_word('多人')
jieba.add_word('多工')
jieba.add_word('人員')
jieba.add_word('系統化')
jieba.add_word('系統化架構')
jieba.add_word('諮詢')
jieba.add_word('複製')
jieba.add_word('防火牆')
jieba.add_word('資料庫')
jieba.add_word('vue.js')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 0.931 seconds.
Prefix dict has been built successfully.


In [21]:
content3 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

In [27]:
for i in range(3, 4):
    print(content2.loc[i, 'content'][:1000])

系統功能 求分析設計 系統功能規劃 架構建構佈署 具出色 溝通能力 組織協調能力 邏輯分析能力強 團隊合作精神 有專案經理經驗 c#工具應用 具軟體專案開發 軟體產品研發 系統分析 系統設計經驗 有uml設計經驗 軟體專案管理 求導引 溝通技巧能力 經驗 各種資料庫整合開發技術 如mssql oracle 任一種資料庫 網站程式開發基本能力 如jsp/asp.net 開發語言 1 nbs升級案 專案 文件 進度管理2 監督 成員 各項 進度回報3 客戶進行各項 求確認4 各階段驗收事宜5 專案各項 求 分派 品質確認 專案溝通╱整合管理 專案管理架構 專案說明 vici holdings 威旭資訊 專注 高頻 造市 套利交易 進行量化研究 追求 好 交易策略 擁有領先全台 軟體研發團隊 華爾街 級 fpga設計技術 據此打造低延遲全自動交易系統 同時 交易策略橫跨股票 期貨 衍生性商品 每日全球交易市值達數百億台幣 以人為本 開放 自由 環境 讓夥伴可以專注 研究 創新 同時 透明 扁平化 組織架構 讓所有人 可以在組織裡發聲 發揮潛能 資深軟體工程師 要針對不同金融交易市場開發專屬 自動交易系統 為了爭奪幾個nanosecond 延遲時間縮短而不斷努力 因此 要高度專業 熱忱 軟體 職責 開發 維護金融交易 系統 研究 導入新技術 開發方法 指導 軟體開發人員 專業能力 求 資訊工程科系大學畢業 同 電腦科學基礎 modern c++ multithread socket 軟體開發技術 了解 linux 軟體開發 有團隊協作 解決問題 能力 能依據 求規格設計軟體架構 有開發高效能即時系統 經驗 追求極限效能有研究熱情 加分項 有金融交易 系統 開發經驗 流程 可能視情況調整流程順序 篩選 coding test 主管 技術長 hr 可選擇現場 google meet 系統架構規劃 軟體程式設計 反洗錢架構 sas aml系統 有帶領團隊經驗 善溝通 有技術程式開發經驗 有專案管理經驗 具金融業資訊系統開發 導入經驗 駐點 洞察 量化系統流程瓶頸 關注服務穩定性 發掘系統 performance security 缺陷 解決方案 在 求快速成長 環境中仍能以數據驅動 方式找到優先級 高 自動化 流程 快速定位系統問題 必須 6 年以上 rd/sre/devops 

In [28]:
content_ = content2.loc[3, 'content'][:1000]

In [22]:
#精確模式（Default Mode）
seg_list = []
for i in range(len(content2)):
    seg_list = jieba.cut(content2.loc[i, 'content'], cut_all=False) #預設是精確模式　seg_list = jieba.cut(test)
    content3.loc[i, 'content'] = "|".join(seg_list)

In [23]:
content3

,category,content
0,職位 I,微控制器 程式 設計 有 興趣 人員 量測 控制 通訊 人機 介面 ...
1,職位 II,網路安控 軟體 系統 開發 系統 測試 熟 pki 專案管理 java ...
2,職位 III,ic 驗證 驗證 系統 開發 客戶 design in 問題 解決 fpga...
3,職位 IV,系統 功能 求 分析 設計 系統 功能 規劃 架構 建構 佈署 具 出色 ...


In [26]:
r_3 = r'\b\d+\b'

In [27]:
content4 = pd.DataFrame({
    'category': ['職位 I', '職位 II', '職位 III', '職位 IV'],
    'content':None
})

for j in range(len(content3)):
    c20 = re.sub(r_3, ' ', content3.loc[j, 'content'])
    c21 = c20.split()
    c_list = []
    i = len(c21)-1
    while i>=0:
        if 'c' not in c21[i]:
            c21[i] = c21[i].replace("#", "")
            c21[i] = c21[i].replace("+", "")
        if len(c21[i]) <= 1:
            i-=1
            continue
        if c21[i] == '.' or c21[i] == '..'or c21[i] == '...' or c21[i] == '....' or c21[i] == '.....' \
        or c21[i] == '......'or c21[i] == '.......'or c21[i] == '........'or c21[i] == '.........'or c21[i] == '..........'\
        or c21[i] == '/' or c21[i] == '//' or c21[i] == '\\' or c21[i] == '\\\\'\
        or (not c21[i].isprintable()):
            i -= 1
            continue
        if c21[i] == 'co' and c21[i-1] == '/' and c21[i-2] == 'fi':
            if c21[i-3] == 'hana':
                c_list.append('s/4 hana fi/co')
                i -= 7
                continue
            elif c21[i-3] == '4' and c21[i-4] == '\/' and c21[i-5] == 's':
                c_list.append('s/4 fi/co')
                i -= 6
                continue
        if c21[i] == 'hana':
            if c21[i-1] == '4' and c21[i-2] == '/' and c21[i-3] == 's':
                i -= 4
                continue
        c_list.append(c21[i])
        i -= 1
    content4.loc[j, 'content'] = c_list[::-1]

In [28]:
content4

,category,content
0,職位 I,"[微控制器, 程式, 設計, 興趣, 人員, 量測, 控制, 通訊, 人機, 介面, 韌體,..."
1,職位 II,"[網路安控, 軟體, 系統, 開發, 系統, 測試, pki, 專案管理, java, as..."
2,職位 III,"[ic, 驗證, 驗證, 系統, 開發, 客戶, design, in, 問題, 解決, f..."
3,職位 IV,"[系統, 功能, 分析, 設計, 系統, 功能, 規劃, 架構, 建構, 佈署, 出色, 溝..."


In [33]:
#tf idf

In [34]:
uniqueWords = set()
for i in range(len(content4)):
    a = content4.loc[i,"content"]
    uniqueWords = uniqueWords.union(set(a))

In [35]:
len(uniqueWords)

61400

In [36]:
# 職缺 I 的 bag of word 統計
numOfWords = dict.fromkeys(uniqueWords, 0)

# 初始資料表
df_por = pd.DataFrame(
{'por':numOfWords}
)


for i in range(len(content4)):
    a = content4.loc[i,"content"]
    for word in a:
        numOfWords[word] += 1

    # 第i職缺
    df = pd.DataFrame({i:numOfWords}) 
    df_por = pd.concat([df_por, df], axis = 1)

    # 初始化
    numOfWords = dict.fromkeys(uniqueWords, 0)


In [37]:
df_por.drop('por', axis=1, inplace=True)
df_por

,0,1,2,3
...............................................,0,2,1,0
.........................................................................,0,3,4,0
000jpy,0,3,0,1
000senior,0,3,0,1
00am,0,0,3,0
...,...,...,...,...
龐大,0,52,6,10
龐大資源,0,4,0,0
龐雜,0,9,2,0
龜山,0,12,16,1


In [63]:
df_por['sum'] = df_por[0] + df_por[1] + df_por[2] + df_por[3]

In [64]:
df_por

,0,1,2,3,sum
...............................................,0,2,1,0,3
.........................................................................,0,3,4,0,7
000jpy,0,3,0,1,4
000senior,0,3,0,1,4
00am,0,0,3,0,3
...,...,...,...,...,...
龐大,0,52,6,10,68
龐大資源,0,4,0,0,4
龐雜,0,9,2,0,11
龜山,0,12,16,1,29


In [65]:
df_por = df_por[df_por['sum']>5]

In [67]:
df_por

,0,1,2,3,sum
.........................................................................,0,3,4,0,7
00pm,0,0,8,0,8
100b,0,4,0,2,6
100k,0,19,4,7,30
10g,0,12,20,8,40
...,...,...,...,...,...
龍潭,0,0,8,1,9
龍頭,0,13,25,2,40
龐大,0,52,6,10,68
龐雜,0,9,2,0,11


In [68]:
df_por[0].to_dict()

{'.........................................................................': 0,
 '00pm': 0,
 '100b': 0,
 '100k': 0,
 '10g': 0,
 '10gbps': 0,
 '10gpon': 0,
 '10x': 0,
 '11a': 0,
 '11ac': 0,
 '11ax': 0,
 '11b': 0,
 '11g': 0,
 '11pm': 0,
 '12c': 1,
 '12cm': 0,
 '16bit': 0,
 '17live': 0,
 '1g': 0,
 '1h': 0,
 '1k': 0,
 '1m': 0,
 '1q': 0,
 '1st': 0,
 '1x': 0,
 '1year': 0,
 '2012r2': 0,
 '2022awsutc': 0,
 '2022digitalent': 0,
 '2022wsutc': 0,
 '2022ybkcgplan': 0,
 '2024q4': 0,
 '20m': 0,
 '20yrs': 0,
 '22301etc': 0,
 '22nd': 0,
 '23year': 0,
 '24h': 0,
 '24hr': 0,
 '24x7': 0,
 '25g': 0,
 '26k': 0,
 '2b': 0,
 '2d': 0,
 '2d.': 0,
 '2g': 0,
 '2m': 0,
 '2nd': 0,
 '2y': 0,
 '2year': 0,
 '2yr': 0,
 '30k': 0,
 '30minute': 0,
 '32bit': 0,
 '32bits': 0,
 '32k': 0,
 '33fbcb16f8ea': 0,
 '33k': 0,
 '34k': 0,
 '35k': 0,
 '36k': 0,
 '38k': 0,
 '3a': 0,
 '3c': 0,
 '3com': 0,
 '3d': 0,
 '3drens': 0,
 '3ds': 0,
 '3dsmax': 0,
 '3g': 0,
 '3gpp': 6,
 '3m': 0,
 '3m2y': 0,
 '3par': 0,
 '3rd': 1,
 '3rdparty': 0,
 

In [69]:
# TF

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [70]:
numOfWords = dict.fromkeys(uniqueWords, 0)

df_TF = pd.DataFrame(
{'TF_por':numOfWords}
)

for i in range(len(content4)):
    a = content4.loc[i,'content']

    # 第i職缺
    df = pd.DataFrame({i:computeTF(df_por[i].to_dict(),a)}) 
    df_TF = pd.concat([df_TF, df], axis = 1)

    # 初始化
    df = dict.fromkeys(uniqueWords, 0)
    


df_TF

,TF_por,0,1,2,3
...............................................,0,NaN,NaN,NaN,NaN
.........................................................................,0,0.0,0.000001,1.870080e-06,0.000000
000jpy,0,NaN,NaN,NaN,NaN
000senior,0,NaN,NaN,NaN,NaN
00am,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
龐大,0,0.0,0.000021,2.805120e-06,0.000020
龐大資源,0,NaN,NaN,NaN,NaN
龐雜,0,0.0,0.000004,9.350400e-07,0.000000
龜山,0,0.0,0.000005,7.480320e-06,0.000002


In [71]:
df_TF.loc['料庫',1]

nan

In [72]:
# df_TF.drop('TF_por', axis=1, inplace=True)
df_TF

,TF_por,0,1,2,3
...............................................,0,NaN,NaN,NaN,NaN
.........................................................................,0,0.0,0.000001,1.870080e-06,0.000000
000jpy,0,NaN,NaN,NaN,NaN
000senior,0,NaN,NaN,NaN,NaN
00am,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
龐大,0,0.0,0.000021,2.805120e-06,0.000020
龐大資源,0,NaN,NaN,NaN,NaN
龐雜,0,0.0,0.000004,9.350400e-07,0.000000
龜山,0,0.0,0.000005,7.480320e-06,0.000002


In [82]:
df_TF = df_TF.dropna()

In [83]:
# IDF
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    return idfDict

In [84]:
df_por_list = [] 

for i in range(len(content4)):
    df_por_list.append(df_por[i].to_dict())


idfs = computeIDF(df_por_list)

In [85]:
df_idfs = pd.DataFrame({'idf':idfs},)
df_idfs.sort_values(by=["idf"],ascending=False)

,idf
功學,0.60206
播映,0.60206
ganglia,0.60206
教用,0.60206
gatsby,0.60206
...,...
possible,0.00000
證產品,0.00000
bringup,0.00000
post,0.00000


In [86]:
df_idfs.loc['資料庫',:]

idf    0.124939
Name: 資料庫, dtype: float64

In [87]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [88]:
df_por

,0,1,2,3,sum
.........................................................................,0,3,4,0,7
00pm,0,0,8,0,8
100b,0,4,0,2,6
100k,0,19,4,7,30
10g,0,12,20,8,40
...,...,...,...,...,...
龍潭,0,0,8,1,9
龍頭,0,13,25,2,40
龐大,0,52,6,10,68
龐雜,0,9,2,0,11


In [ ]:
df_por = df_por.drop('sum', axis=1)
df_por

In [92]:
tfidf_list = []
for i in range(len(df_por.columns)):
    tfidf = computeTFIDF(df_TF[i].to_dict(), idfs)
    tfidf_list.append(tfidf)

df_tfidf = pd.DataFrame(tfidf_list).T

df_tfidf

,0,1,2,3
.........................................................................,0.0,3.605075e-07,5.629502e-07,0.000000e+00
00pm,0.0,0.000000e+00,2.251801e-06,0.000000e+00
100b,0.0,4.806766e-07,0.000000e+00,1.205296e-06
100k,0.0,9.476194e-07,2.336454e-07,1.750851e-06
10g,0.0,5.984965e-07,1.168227e-06,2.000973e-06
...,...,...,...,...
龍潭,0.0,0.000000e+00,1.125900e-06,6.026482e-07
龍頭,0.0,6.483712e-07,1.460284e-06,5.002432e-07
龐大,0.0,2.593485e-06,3.504681e-07,2.501216e-06
龐雜,0.0,1.081522e-06,2.814751e-07,0.000000e+00


In [98]:
dic_0 = df_tfidf.sort_values([0], ascending=False)[0].head(150).to_dict()
dic_1 = df_tfidf.sort_values([1], ascending=False)[1].head(100).to_dict()
dic_2 = df_tfidf.sort_values([2], ascending=False)[2].head(100).to_dict()
dic_3 = df_tfidf.sort_values([3], ascending=False)[3].head(100).to_dict()

In [99]:
with open('tfidf_0.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    for row in dic_0.items():
        writer.writerow(row)

In [95]:
with open('tfidf_1.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    for row in dic_1.items():
        writer.writerow(row)

In [96]:
with open('tfidf_2.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    for row in dic_2.items():
        writer.writerow(row)

In [97]:
with open('tfidf_3.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    for row in dic_3.items():
        writer.writerow(row)